In [1]:
using Pkg
Pkg.activate("..")
using Revise
using FUSE

  Activating environment at `~/.julia/dev/FUSE/Project.toml`
┌ Info: Precompiling FUSE [e64856f0-3bb8-4376-b4b7-c03396503992]
└ @ Base loading.jl:1317
┌ Warning: Package FUSE does not have Pkg in its dependencies:
│ - If you have FUSE checked out for development and have
│   added Pkg as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with FUSE
└ Loading Pkg into FUSE from project dependency, future warnings for FUSE are suppressed.
  Activating environment at `~/.julia/dev/FUSE/Project.toml`


In [4]:
cp = FUSE.core_profiles()
cp.profiles_1d[1].grid.rho_tor_norm = range(0, 1, length=10)
cp.profiles_1d[1].electrons.density = (1.0 .- cp.profiles_1d[1].grid.rho_tor_norm).^2
length(cp.profiles_1d[1].grid.rho_tor_norm) == 10

# resize an array of struct
resize!(cp.profiles_1d, 2)
length(cp.profiles_1d) == 2
cp.profiles_1d[1]=cp.profiles_1d[2]

LoadError: UndefRefError: access to undefined reference

In [18]:
abstract type bla{T} <: AbstractArray{T,1} end

In [2]:
using StructArrays
using JSON

"""
Resize StructArray to contain n elements. If n is smaller than the current collection length, the first n elements will be retained.
If n is larger, the new elements are not guaranteed to be initialized.
"""
function resize!(collection::StructArray, n::Int)
    if n>length(collection)
        for k in length(collection):n-1
            push!(collection,eltype(collection)())
            # println("push $(length(collection))")
        end
    elseif n<length(collection)
        for k in n:length(collection)-1
            pop!(collection)
            # println("pop $(length(collection))")
        end
    end
    return collection
end

Base.@kwdef struct c
    d:: Vector{Float64} = [1.0, 2.0, 3.0]
end

Base.@kwdef struct b
    c :: StructArray{c} = StructArray(c() for k in 1:1)#; unwrap = t -> t<:c)
end

Base.@kwdef struct a
    b :: StructArray{b} = StructArray(b() for k in 1:1)#; unwrap = t -> t<:b)
end

tmp=a()
resize!(tmp.b,3)
resize!(tmp.b,4)
resize!(tmp.b,2)
resize!(tmp.b,0)
resize!(tmp.b,5)
JSON.print(tmp,4)

{
    "b": [
        {
            "c": [
                {
                    "d": [
                        1.0,
                        2.0,
                        3.0
                    ]
                }
            ]
        },
        {
            "c": [
                {
                    "d": [
                        1.0,
                        2.0,
                        3.0
                    ]
                }
            ]
        },
        {
            "c": [
                {
                    "d": [
                        1.0,
                        2.0,
                        3.0
                    ]
                }
            ]
        },
        {
            "c": [
                {
                    "d": [
                        1.0,
                        2.0,
                        3.0
                    ]
                }
            ]
        },
        {
            "c": [
                {
                    "d": [
             

In [13]:
Complex{Float64}

ComplexF64 (alias for Complex{Float64})

In [33]:
struct test3
    @reserved const::Int
end
tmp=test3(0.0)
tmp.const_

LoadError: syntax: expected assignment after "const"

0